In [185]:
# -*- coding:utf-8 -*-
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import *
import os


# train/validation/test 要根据时间点分，否则会有leakage
df = pd.read_csv("pump_sample_raw.csv")

df.loc[807, "pre_3d_market_cap_btc"] = 140.867
df.loc[807, "pre_3d_market_cap_usd"] = 1377987
df.loc[1285, "pre_3d_market_cap_usd"] = 1377987
df.loc[1285, "pre_3d_market_cap_btc"] = 140.867
df.loc[1247, "pre_3d_market_cap_usd"] = 2707118
df.loc[1247, "pre_3d_market_cap_btc"] = 225.352
df.loc[1248, "pre_3d_market_cap_usd"] = 2625736
df.loc[1248, "pre_3d_market_cap_btc"] = 231.999

for i in [102, 243, 415, 515, 580, 600, 872, 1056, 1094, 1314, 1331]:
    df.loc[i, "pre_3d_market_cap_btc"] = 279.966
    df.loc[i, "pre_3d_market_cap_usd"] = 2036283
df.loc[480, "pre_3d_market_cap_btc"] = 836.675
df.loc[480, "pre_3d_market_cap_usd"] = 8467606

df = df[df["exchange"] == "binance"]
df = df[df["pair"] == "BTC"]



In [186]:
df.shape

(981, 154)

In [187]:
#其余的使用平均值填充
for column in ['pre_3d_market_cap_usd','pre_3d_market_cap_btc', 'pre_3d_price_usd', 'pre_3d_price_btc','pre_3d_volume_usd', 'pre_3d_volume_btc', 'pre_3d_twitter_index','pre_3d_reddit_index', 'pre_3d_alexa_index']:
    mean_val = df[column].mean()
    df[column].fillna(mean_val, inplace=True)


price_columns = []
return_columns = []
volume_columns = []


for column in df.columns:
    if "price" in column:
        price_columns.append(column)
    if "return" in column:
        return_columns.append(column)
    if "volume" in column:
        volume_columns.append(column)

other_columns = ['pre_3d_alexa_index', 'pre_3d_market_cap_btc', 'pre_3d_market_cap_usd', 'pre_3d_reddit_index', 'pre_3d_twitter_index']

for column in price_columns:
    df[column] = df[column] * (10**5)

for column in volume_columns + other_columns:
    df[column] = np.log2(df[column]+0.1)


In [188]:
# sequence 化
for idx, row in df.iterrows():
    feature = []
    for column in price_columns + return_columns + volume_columns:
        feature.append(row[column])

    feature_str = "".join(map(lambda x: str(x), feature))
    df.loc[idx, "feature"] = feature_str

In [189]:
feature

[1.579,
 1.584,
 20.0,
 10.792,
 10.0275,
 1.569946404183088,
 1.569946404183088,
 1.571091800918009,
 1.576342735259163,
 1.5992689232700283,
 1.5751780663847261,
 1.5496948815947236,
 1.5850057320039062,
 1.5463929529549845,
 1.5597374524828302,
 1.6665877777270164,
 1.620817245771978,
 1.3731535484031565,
 1.5985723017363078,
 1.1629223140626481,
 1.5205698730133343,
 1.28265475353488,
 1.4767958405471984,
 41197.32921073641,
 1.3548580309997447,
 12.666244458518053,
 12.626262626262626,
 1.853224610822832,
 0.9291604892512972,
 -0.0113625378532722,
 0.0027857451450921,
 -0.0158174464222258,
 0.0002277394481182,
 -0.0145085901901662,
 -0.0055347264127978,
 0.1592533248107106,
 0.2307385802401527,
 0.2163564489035738,
 14.645382609941212,
 14.645382609941212,
 14.645382609941212,
 11.32479935481874,
 -0.9929441812058931,
 -2.8318557544288194,
 13.439714030298736,
 14.722297019903,
 16.307255962420452,
 13.014735362029231,
 13.33410099346632,
 14.91905418052881,
 -1.86457797612487,
 -

In [190]:
df.feature

0       1.13951.1831.1921.17351.1651.140184730238...
1       0.3110.3160.3280.3210.3260.30959001143345...
2       1.2211.4371.4781.42649999999999991.3551.2...
3       0.1130.1150.1370.1260.130.111957424614580...
4       0.1020.1020.170.11750.11750.1020166336029...
                              ...                        
1330    89.08595.0996.7993.2250000000000191.26588...
1331    1.43451.4321.791.611.47851.42944122460615...
1332    1.3992.0092.0581.93850000000000011.90851....
1333    2.44952.45799999999999972.9942.65152.3725...
1334    1.5791.58420.010.79210.02751.569946404183...
Name: feature, Length: 981, dtype: object

In [21]:
column_list = list(df.columns)
column_list

['channel_id',
 'session_id',
 'coin',
 'exchange',
 'pair',
 'timestamp',
 'timestamp_unix',
 'insider_price',
 'outsider_price',
 'highest_price',
 'outsider_avg_price',
 'outsider_next_avg_price',
 'insider_return',
 'outsider_return',
 'outsider_avg_return',
 'outsider_next_avg_return',
 'pre_1h_return',
 'pre_1h_price',
 'pre_1h_price_avg',
 'pre_1h_volume',
 'pre_1h_volume_avg',
 'pre_1h_volume_sum',
 'pre_1h_volume_tb',
 'pre_1h_volume_quote',
 'pre_1h_volume_quote_tb',
 'pre_3h_return',
 'pre_3h_price',
 'pre_3h_price_avg',
 'pre_3h_volume',
 'pre_3h_volume_avg',
 'pre_3h_volume_sum',
 'pre_3h_volume_tb',
 'pre_3h_volume_tb_avg',
 'pre_3h_volume_tb_sum',
 'pre_3h_volume_quote',
 'pre_3h_volume_quote_avg',
 'pre_3h_volume_quote_sum',
 'pre_3h_volume_quote_tb',
 'pre_3h_volume_quote_tb_avg',
 'pre_3h_volume_quote_tb_sum',
 'pre_6h_return',
 'pre_6h_price',
 'pre_6h_price_avg',
 'pre_6h_volume',
 'pre_6h_volume_avg',
 'pre_6h_volume_sum',
 'pre_6h_volume_tb',
 'pre_6h_volume_tb_av

In [ ]:
["insider_price, outsider_price, highest_price, outsider_avg_price, outsider_next_avg_price, insider_return, outsider_return, outsider_avg_return, outsider_next_avg_return]

In [ ]:
pre_feature_columns = []

In [191]:
df.head(5)

,channel_id,session_id,coin,exchange,pair,timestamp,timestamp_unix,insider_price,outsider_price,highest_price,...,pre_3d_market_cap_usd,pre_3d_market_cap_btc,pre_3d_price_usd,pre_3d_price_btc,pre_3d_volume_usd,pre_3d_volume_btc,pre_3d_twitter_index,pre_3d_reddit_index,pre_3d_alexa_index,feature
0,1095533634,256,APPC,binance,BTC,2019-06-04 12:01:33+00:00,1559649693000,1.1395,1.183,1.192,...,23.462615,10.396219,10859.732678,1.265950,19.915227,6.849997,14.642227,11.721142,20.185433,1.13951.1831.1921.17351.1651.140184730238...
1,1095533634,258,REQ,binance,BTC,2019-06-08 13:00:02+00:00,1559998802000,0.3110,0.316,0.328,...,23.902450,10.997715,2150.080909,0.279940,18.972784,6.067950,15.351426,14.873929,18.755739,0.3110.3160.3280.3210.3260.30959001143345...
2,1095533634,278,ADX,binance,BTC,2019-07-05 17:01:12+00:00,1562346072000,1.2210,1.437,1.478,...,23.540070,10.166876,13256.718795,1.248871,19.269264,5.897873,15.725609,11.923365,19.751394,1.2211.4371.4781.42649999999999991.3551.2...
3,1095533634,285,SNM,binance,BTC,2019-08-07 16:59:48+00:00,1565197188000,0.1130,0.115,0.137,...,22.274391,8.877033,1187.431576,0.110004,15.887122,2.514596,14.925976,13.279915,20.127763,0.1130.1150.1370.1260.130.111957424614580...
4,1095533634,287,DATA,binance,BTC,2019-08-15 20:00:07+00:00,1565899207000,0.1020,0.102,0.170,...,25.147164,11.213932,171097.821294,6.501331,22.216036,8.339730,15.312302,12.989296,20.127763,0.1020.1020.170.11750.11750.1020166336029...


In [192]:
df

,channel_id,session_id,coin,exchange,pair,timestamp,timestamp_unix,insider_price,outsider_price,highest_price,...,pre_3d_market_cap_usd,pre_3d_market_cap_btc,pre_3d_price_usd,pre_3d_price_btc,pre_3d_volume_usd,pre_3d_volume_btc,pre_3d_twitter_index,pre_3d_reddit_index,pre_3d_alexa_index,feature
0,1095533634,256,APPC,binance,BTC,2019-06-04 12:01:33+00:00,1559649693000,1.1395,1.183,1.192,...,23.462615,10.396219,10859.732678,1.265950,19.915227,6.849997,14.642227,11.721142,20.185433,1.13951.1831.1921.17351.1651.140184730238...
1,1095533634,258,REQ,binance,BTC,2019-06-08 13:00:02+00:00,1559998802000,0.3110,0.316,0.328,...,23.902450,10.997715,2150.080909,0.279940,18.972784,6.067950,15.351426,14.873929,18.755739,0.3110.3160.3280.3210.3260.30959001143345...
2,1095533634,278,ADX,binance,BTC,2019-07-05 17:01:12+00:00,1562346072000,1.2210,1.437,1.478,...,23.540070,10.166876,13256.718795,1.248871,19.269264,5.897873,15.725609,11.923365,19.751394,1.2211.4371.4781.42649999999999991.3551.2...
3,1095533634,285,SNM,binance,BTC,2019-08-07 16:59:48+00:00,1565197188000,0.1130,0.115,0.137,...,22.274391,8.877033,1187.431576,0.110004,15.887122,2.514596,14.925976,13.279915,20.127763,0.1130.1150.1370.1260.130.111957424614580...
4,1095533634,287,DATA,binance,BTC,2019-08-15 20:00:07+00:00,1565899207000,0.1020,0.102,0.170,...,25.147164,11.213932,171097.821294,6.501331,22.216036,8.339730,15.312302,12.989296,20.127763,0.1020.1020.170.11750.11750.1020166336029...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,1401539056,5,ZEN,binance,BTC,2020-04-03 19:00:01+00:00,1585940401000,89.0850,95.090,96.790,...,25.568932,12.918803,568383.578863,88.656747,20.960189,8.314296,15.844120,12.316593,15.295163,89.08595.0996.7993.2250000000000191.26588...
1331,1401539056,7,ARN,binance,BTC,2020-04-07 18:00:06+00:00,1586282406000,1.4345,1.432,1.790,...,20.957507,8.129623,171097.821294,6.501331,22.216036,8.339730,15.312302,12.989296,20.127763,1.43451.4321.791.611.47851.42944122460615...
1332,1401539056,9,EDO,binance,BTC,2020-04-10 18:00:09+00:00,1586541609000,1.3990,2.009,2.058,...,25.147164,11.213932,171097.821294,6.501331,22.216036,8.339730,15.312302,12.989296,20.127763,1.3992.0092.0581.93850000000000011.90851....
1333,1325010971,5,MDA,binance,BTC,2021-01-31 00:01:23+00:00,1612051283000,2.4495,2.458,2.994,...,23.497406,8.613085,60421.520979,1.991553,19.507800,4.624778,15.312302,8.531771,21.642455,2.44952.45799999999999972.9942.65152.3725...


In [ ]:
df

In [193]:
X = pd.merge(left=df, right=df[["channel_id","coin","feature", "timestamp"]], how='left', on=["channel_id"],
             sort=False)

In [194]:
X.head(5)

,channel_id,session_id,coin_x,exchange,pair,timestamp_x,timestamp_unix,insider_price,outsider_price,highest_price,...,pre_3d_price_btc,pre_3d_volume_usd,pre_3d_volume_btc,pre_3d_twitter_index,pre_3d_reddit_index,pre_3d_alexa_index,feature_x,coin_y,feature_y,timestamp_y
0,1095533634,256,APPC,binance,BTC,2019-06-04 12:01:33+00:00,1559649693000,1.1395,1.183,1.192,...,1.26595,19.915227,6.849997,14.642227,11.721142,20.185433,1.13951.1831.1921.17351.1651.140184730238...,APPC,1.13951.1831.1921.17351.1651.140184730238...,2019-06-04 12:01:33+00:00
1,1095533634,256,APPC,binance,BTC,2019-06-04 12:01:33+00:00,1559649693000,1.1395,1.183,1.192,...,1.26595,19.915227,6.849997,14.642227,11.721142,20.185433,1.13951.1831.1921.17351.1651.140184730238...,REQ,0.3110.3160.3280.3210.3260.30959001143345...,2019-06-08 13:00:02+00:00
2,1095533634,256,APPC,binance,BTC,2019-06-04 12:01:33+00:00,1559649693000,1.1395,1.183,1.192,...,1.26595,19.915227,6.849997,14.642227,11.721142,20.185433,1.13951.1831.1921.17351.1651.140184730238...,ADX,1.2211.4371.4781.42649999999999991.3551.2...,2019-07-05 17:01:12+00:00
3,1095533634,256,APPC,binance,BTC,2019-06-04 12:01:33+00:00,1559649693000,1.1395,1.183,1.192,...,1.26595,19.915227,6.849997,14.642227,11.721142,20.185433,1.13951.1831.1921.17351.1651.140184730238...,SNM,0.1130.1150.1370.1260.130.111957424614580...,2019-08-07 16:59:48+00:00
4,1095533634,256,APPC,binance,BTC,2019-06-04 12:01:33+00:00,1559649693000,1.1395,1.183,1.192,...,1.26595,19.915227,6.849997,14.642227,11.721142,20.185433,1.13951.1831.1921.17351.1651.140184730238...,DATA,0.1020.1020.170.11750.11750.1020166336029...,2019-08-15 20:00:07+00:00


In [195]:
X = X[X.timestamp_x > X.timestamp_y]

In [196]:
X.head(5)

,channel_id,session_id,coin_x,exchange,pair,timestamp_x,timestamp_unix,insider_price,outsider_price,highest_price,...,pre_3d_price_btc,pre_3d_volume_usd,pre_3d_volume_btc,pre_3d_twitter_index,pre_3d_reddit_index,pre_3d_alexa_index,feature_x,coin_y,feature_y,timestamp_y
14,1095533634,258,REQ,binance,BTC,2019-06-08 13:00:02+00:00,1559998802000,0.311,0.316,0.328,...,0.279940,18.972784,6.067950,15.351426,14.873929,18.755739,0.3110.3160.3280.3210.3260.30959001143345...,APPC,1.13951.1831.1921.17351.1651.140184730238...,2019-06-04 12:01:33+00:00
28,1095533634,278,ADX,binance,BTC,2019-07-05 17:01:12+00:00,1562346072000,1.221,1.437,1.478,...,1.248871,19.269264,5.897873,15.725609,11.923365,19.751394,1.2211.4371.4781.42649999999999991.3551.2...,APPC,1.13951.1831.1921.17351.1651.140184730238...,2019-06-04 12:01:33+00:00
29,1095533634,278,ADX,binance,BTC,2019-07-05 17:01:12+00:00,1562346072000,1.221,1.437,1.478,...,1.248871,19.269264,5.897873,15.725609,11.923365,19.751394,1.2211.4371.4781.42649999999999991.3551.2...,REQ,0.3110.3160.3280.3210.3260.30959001143345...,2019-06-08 13:00:02+00:00
42,1095533634,285,SNM,binance,BTC,2019-08-07 16:59:48+00:00,1565197188000,0.113,0.115,0.137,...,0.110004,15.887122,2.514596,14.925976,13.279915,20.127763,0.1130.1150.1370.1260.130.111957424614580...,APPC,1.13951.1831.1921.17351.1651.140184730238...,2019-06-04 12:01:33+00:00
43,1095533634,285,SNM,binance,BTC,2019-08-07 16:59:48+00:00,1565197188000,0.113,0.115,0.137,...,0.110004,15.887122,2.514596,14.925976,13.279915,20.127763,0.1130.1150.1370.1260.130.111957424614580...,REQ,0.3110.3160.3280.3210.3260.30959001143345...,2019-06-08 13:00:02+00:00


In [197]:
def udf(df):
    def takeFirst(elem):
        return elem[0]

    # output = []
    feature_seq = []
    coin_seq = []

    X = list(zip(df.timestamp_y, df.coin_y, df.feature_y))
                 # , df.director, df.actor))

    X.sort(key=takeFirst, reverse=True)
    length = 0
    for x in X:  # set max length to 100
        coin_seq.append(str(x[1]))
        feature_seq.append(str(x[2]))

        length += 1
        if length >= 50:
            break

    return np.array([[df.iloc[0]["channel_id"], df.iloc[0]["coin_x"], df.iloc[0]["timestamp_x"], df.iloc[0]["session_id"],
                      str(length), "\t".join(coin_seq), "\t".join(feature_seq)]])


In [41]:
X.iloc[0]["channel_id"]

1095533634

In [199]:
X_ = X.groupby(["channel_id", "coin_x", "timestamp_x", "session_id"]).apply(udf)

In [200]:
ui_sample_base = pd.DataFrame(np.concatenate(X_.values, axis=0), columns=["channel_id", "coin", "timestamp", "session_id", "length", "coin_seq", "feature_seq"])


In [205]:
ui_sample_base.loc[0, "coin_seq"].split("\t")

['POA', 'WABI', 'OAX']

In [208]:
ui_sample_base.columns

Index(['channel_id', 'coin', 'timestamp', 'session_id', 'length', 'coin_seq',
       'feature_seq'],
      dtype='object')

In [56]:
ui_sample_base.loc[0,"feature_seq"].split("\t")[0].split("")

['POA',
 '0.14300000000000002',
 '0.19999999999999998',
 '0.21',
 '0.1875',
 '0.17600000000000002',
 '0.1516054898315714',
 '0.1516054898315714',
 '0.14184312287882314',
 '0.14497904204498674',
 '0.1353126697002866',
 '0.14092396605155377',
 '0.1323572529628285',
 '0.135897411880108',
 '0.12661868613053626',
 '0.1323466372332273',
 '0.12098271629852855',
 '0.1269597531611803',
 '0.11188052507187318',
 '0.12380329530590133',
 '0.0980285476323547',
 '0.11998145155438737',
 '0.09642119298372057',
 '0.11612295197108254',
 '3269.3874366541504',
 '0.09599991950746277',
 '1.4685314685314683',
 '1.05',
 '1.12',
 '0.9386666666666668',
 '0.0715038709654656',
 '0.094568859781694',
 '0.1110227777432097',
 '0.1343700523678834',
 '0.2217348315657072',
 '0.2660600331772187',
 '0.3232888436858376',
 '0.5467126139179681',
 '0.5842986029499753',
 '22.63016736651883',
 '22.63016736651883',
 '22.63016736651883',
 '22.373986125216415',
 '3.3131290932657436',
 '3.0625055849224907',
 '21.31365702343211',
 '2

In [207]:
ui_sample_base.to_csv("pos_sample_fg.csv", index=False, header=False)

In [59]:
ui_sample_base

,channel_id,coin,timestamp,session_id,length,feature_seq
0,1068531379,DREP,2021-07-25 17:00:01+00:00,74,3,POA0.143000000000000020.199999999999999980....
1,1068531379,EVX,2021-10-03 16:49:26+00:00,99,9,NXS1.3631.3791.85599999999999991.38551.62...
2,1068531379,EVX,2021-10-24 17:00:07+00:00,109,12,WNXM117.5117.8162.1139.95132.6117.571588...
3,1068531379,EZ,2021-08-08 17:00:10+00:00,79,4,DREP1.93400000000000022.4142.8592.4962.64...
4,1068531379,EZ,2021-10-03 17:00:07+00:00,99,10,EVX1.2781.3061.62499999999999981.4171.460...
...,...,...,...,...,...,...
883,1549584841,MDA,2021-11-23 15:00:06+00:00,32,3,NAS0.840.88999999999999991.421.1551.16000...
884,1549584841,NAS,2021-11-14 15:00:08+00:00,25,2,EZ8.47958.51799999999999914.899999999999999...
885,1549584841,NEBL,2021-12-05 15:00:02+00:00,41,5,PHB1.2651.26900000000000012.2181.743500000...
886,1549584841,PHB,2021-11-28 17:00:06+00:00,37,4,MDA1.60999999999999991.6052.2111.9081.811...


In [209]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
dataset = tf.data.TextLineDataset("pos_sample_fg.csv")

In [210]:
def parse_split(line):
    parse_res = tf.string_split([line], delimiter=",")
    values = parse_res.values
    channel = values[0]
    coin = values[1]
    length = values[4]
    coin_seq = values[5]
    feature_seq = values[6]

    return channel, coin, coin_seq, feature_seq, length

dataset = dataset.map(parse_split, num_parallel_calls=2)

In [211]:
dataset = dataset.shuffle(3).repeat(10).batch(10)
iterator = dataset.make_one_shot_iterator()

channel, coin, coin_seq, feature_seq, length = iterator.get_next()

In [212]:
sess = tf.Session()

In [215]:
sess.run([coin_seq, channel])

[array([b'FXS\tVIB\tNAS\tEZ\tDREP\tPOA\tWABI\tOAX',
        b'EZ\tEVX\tWNXM\tEZ\tEVX\tNXS\tFXS\tVIB\tNAS\tEZ\tDREP\tPOA\tWABI\tOAX',
        b'NAS\tEZ\tDREP\tPOA\tWABI\tOAX',
        b'EZ\tEVX\tNXS\tFXS\tVIB\tNAS\tEZ\tDREP\tPOA\tWABI\tOAX',
        b'WABI\tOAX', b'NEBL', b'OAX',
        b'BQX\tPOWR\tPOLY\tEVX\tSUB\tBNT\tNEBL',
        b'STORJ\tLRC\tBNT\tNAV\tBQX\tBQX\tPOWR\tPOLY\tEVX\tSUB\tBNT\tNEBL',
        b'RLC\tELF\tRLC\tNXS\tBRD\tSTORJ\tLRC\tBNT\tNAV\tBQX\tBQX\tPOWR\tPOLY\tEVX\tSUB\tBNT\tNEBL'],
       dtype=object),
 array([b'1068531379', b'1068531379', b'1068531379', b'1068531379',
        b'1068531379', b'1093396548', b'1068531379', b'1093396548',
        b'1093396548', b'1093396548'], dtype=object)]

In [216]:
def parse_sequence(self, sequence):
    """
    split the sequence and convert to dense tensor
    """
    split_sequence = tf.string_split(sequence, delimiter="\t")

    split_sequence = tf.string_split(sequence, delimiter="")
    split_sequence = tf.sparse_to_dense(sparse_indices=split_sequence.indices,
                                        output_shape=[self.feat_config["batch_size"],
                                                      self.feat_config["max_length"]],
                                        sparse_values=split_sequence.values, default_value="0")

    return split_sequence

In [226]:
split_sequence = tf.string_split(feature_seq, delimiter="\t")

In [227]:
split_sequence = tf.sparse_to_dense(sparse_indices=split_sequence.indices,
                                        output_shape=[10,50],
                                        sparse_values=split_sequence.values, default_value="".join(["0" for i in range(142)]))

In [228]:
split_sequence1 = tf.reshape(split_sequence, shape=[10*50])

In [229]:
split_sequence2 = tf.string_split(split_sequence1, delimiter="")

In [230]:
split_sequence3 = tf.sparse_to_dense(sparse_indices=split_sequence2.indices,
                                        output_shape=[500,142],
                                        sparse_values=split_sequence2.values, default_value="0")

In [231]:
sess.run(split_sequence3)

array([[b'117.5', b'117.8', b'162.1', ..., b'3.9996246926820067',
        b'22.57389374235034', b'6.815648467619014'],
       [b'13.9885', b'14.96', b'21.0', ..., b'5.120037885260384',
        b'20.17646439730832', b'4.838390192370911'],
       [b'1.363', b'1.379', b'1.8559999999999999', ...,
        b'3.196560355600302', b'18.18257922027563',
        b'2.8605947439060366'],
       ...,
       [b'0', b'0', b'0', ..., b'0', b'0', b'0'],
       [b'0', b'0', b'0', ..., b'0', b'0', b'0'],
       [b'0', b'0', b'0', ..., b'0', b'0', b'0']], dtype=object)

In [165]:
split_sequence_final = tf.reshape(split_sequence3,shape=[10,50,143])

In [168]:
v1, v2 = sess.run([split_sequence_final, feature_seq])

In [169]:
v1

array([[[b'BRD', b'0.448', b'0.44499999999999995', ...,
         b'4.430998863818386', b'19.64450554632525',
         b'4.0725639832326355'],
        [b'NAS', b'1.26', b'1.32', ..., b'4.18817800894118',
         b'22.74145282987838', b'7.283317777335898'],
        [b'NXS', b'1.4845', b'1.398', ..., b'3.926971723565651',
         b'16.979180637602493', b'1.7355101643054751'],
        ...,
        [b'0', b'0', b'0', ..., b'0', b'0', b'0'],
        [b'0', b'0', b'0', ..., b'0', b'0', b'0'],
        [b'0', b'0', b'0', ..., b'0', b'0', b'0']],

       [[b'SKY', b'2.5475', b'2.666', ..., b'7.2235786398584025',
         b'20.578363145815093', b'5.521298736329534'],
        [b'PNT', b'1.579', b'1.584', ..., b'5.985151031288507',
         b'21.095865138370375', b'6.20569956180361'],
        [b'DLT', b'0.15', b'0.149', ..., b'3.2164536684601543',
         b'18.539915586778513', b'3.584079734428059'],
        ...,
        [b'0', b'0', b'0', ..., b'0', b'0', b'0'],
        [b'0', b'0', b'0', ..., 

In [180]:
"1,2,3".split(",")

['1', '2', '3']

In [184]:
v2[0]

b'BRD\x020.448\x020.44499999999999995\x020.681\x020.5630000000000001\x020.4855\x020.44729869256034965\x020.44729869256034965\x020.44744606673173704\x020.4460178288240199\x020.44159130879345604\x020.4447251047888988\x020.4374648838328813\x020.441673002028654\x020.43703845521561563\x020.43683113784564015\x020.3970825709847242\x020.42957645936807776\x020.39047473666886107\x020.42169633796424777\x020.3709923771351513\x020.41381430749460085\x020.3786356798992741\x020.40742630535293906\x0219299.02014737333\x020.3938446203777358\x021.5200892857142856\x021.5303370786516854\x021.2095914742451157\x020.8623445825932504\x020.0090004214574299\x020.0028102600059265\x020.0158770245678114\x020.0293233198338422\x020.0383985742167705\x020.1250094793870007\x020.1247838353753632\x020.2012732194686917\x020.1622451365170878\x0217.25852094896545\x0217.25852094896545\x0217.25852094896545\x0216.825059393627605\x02-0.3190426527333255\x02-0.6907065153542208\x0219.38229282216847\x0218.313587667123926\x0219.898549